# imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# constants

In [2]:
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NEURONS = 128

# fetch

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# prepare

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

# train

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


/Users/jtucker/Desktop/working2/keras-by-example/.environment/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-03-19 19:36:54.977035: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6043 - sparse_categorical_accuracy: 0.8347 - val_loss: 0.1924 - val_sparse_categorical_accuracy: 0.9431
Epoch 2/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1737 - sparse_categorical_accuracy: 0.9502 - val_loss: 0.1365 - val_sparse_categorical_accuracy: 0.9594
Epoch 3/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1183 - sparse_categorical_accuracy: 0.9671 - val_loss: 0.1109 - val_sparse_categorical_accuracy: 0.9667
Epoch 4/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0873 - sparse_categorical_accuracy: 0.9743 - val_loss: 0.1000 - val_sparse_categorical_accuracy: 0.9699
Epoch 5/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0727 - sparse_categorical_accuracy: 0.9804 - val_loss: 0.0885 - val_sparse_categorical_accuracy: 0.9739
Epoch 6/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0630 - sparse_categorical_accuracy: 0.9825 - val_loss: 0.0816 - val_sparse_categorical_accuracy: 0.9745
